*Verified by Leila Belabbassi (to work with Pangeo), July 12, 2018*

In [26]:
import xarray as xr
import pandas as pd
import pickle as pk
import re
import requests
import os
import gc
#!pip install netcdf4 == 1.5.0

In [31]:
# Sensor E: url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T104012-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181020T213838-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# Sensor C url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T041943-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [32]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]
#datasets

In [33]:
print(datasets[1])

https://opendap.oceanobservatories.org/thredds/dodsC/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_20191006T180000-20191014T115959.950000.nc#fillmismatch


In [34]:
ds = xr.open_dataset(datasets[1])

In [ ]:
ds = xr.open_mfdataset(datasets)
#ds = ds.swap_dims({'obs': 'time'}) # Swap the primary dimension

In [11]:
ds = xr.open_dataset(datasets[1])

OSError: [Errno -45] NetCDF: Not a valid data type or _FillValue type mismatch: b'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/deployment0001_RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample_20191006T180000-20191014T115959.950000.nc'

In [ ]:
ds

In [ ]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/15s_despike/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [ ]:
# read in the data directly off THREDDS and write out as subsampled pickled pandas dataframe
# NOTE: It takes about one hour to subsample 69499.81 Mbytes of data and write it out to a dataframe. 
num = 0
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})
    
    weird_data = np.array(ds['bottom_pressure'])
    time_array = np.array(ds['time'])

    cleaner_data = []
    cleaner_time = []
    length = len(weird_data)

    idi = 0
    while idi < length:
        idj= idi+10
            while idj < length:
                if np.abs(weird_data[idi] - weird_data[idj]) < .1:
                    cleaner_data.append(weird_data[idj])
                    cleaner_time.append(time_array[idj])
            break
               else:
                idj += 1
                idi = idj

    ds['bp_cleaned'] = cleaner_data
    ds['time_cleaned'] = cleaner_time
    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('.25T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = '/home/jovyan/data/botpt/15s_despikeF/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
    num = num +1

    with open(out, 'wb') as fh:
        pk.dump(pressure_min,fh)

    gc.collect()

In [ ]:
# create a single file with all the pickled data.
pressure_min = pd.DataFrame()
for path, subdirs, files in os.walk('/home/jovyan/data/botpt/1day_mean_dataF/'):
    for name in files:
        file_name = os.path.join(path, name) 
        with open(file_name, 'rb') as f:
            pd_df = pk.load(f)
            pressure_min = pressure_min.append(pd_df)

with open('/home/jovyan/data/botpt/bottom_pressure1day_F11.pkl', 'wb') as f:
    pk.dump(pressure_min,f)